In [11]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import glob
import pickle

In [32]:
#importlib code just for debugging : to allow changes in graphing_config to 
#reflect here. It can be removed if you are not making changes to graphing_config

import importlib ##can remove
import graphing_config as GC
import or_common as ORC
importlib.reload(GC) ##can remove
importlib.reload(orc) ##can remove

<module 'or_common' from '/gpfs/gibbs/pi/reilly/VariantEffects/scripts/noon_scripts/5.graphs/odds_ratio/or_common.py'>

Below cells should be unified between this and the non-odds-ratio notebook

First, let's load the data. 

In [13]:
data={}


for cell_type in GC.cell_types:
    data[cell_type]=pd.read_csv(glob.glob(GC.data_base_path+"malinois_"+cell_type+"/*.csv")[0])


Next, let's get rid of those dumb character substitutions I had to make to appease pyspark

In [14]:
for cell_type in data.keys():
    data[cell_type]=data[cell_type].rename(columns={i:i.replace('^', ',').replace('&','.') for i in data[cell_type].columns})

Currently, whether a given row corresponds to variants that fall within a particular reference activity or skew bin is stored one-hot. 

I want this to be a single column with a categorical value. Let's do that & drop the old one-hot columns.

In [15]:
for cell_type in cell_types:

    current=data[cell_type]

    skew_columns=[]
    ref_columns=[]

    for i in current.columns:
        if "_skew" in i :
            skew_columns.append(i)
        elif "_ref" in i :
            ref_columns.append(i)

    current["ref_bin"]=current.apply(GC.find_true_column, columns=ref_columns, default='Error', axis=1)
    current=current.drop(ref_columns,axis=1)

    current["skew_bin"]=current.apply(GC.find_true_column, columns=skew_columns, default='Error', axis=1)
    current=current.drop(skew_columns,axis=1)
    
    data[cell_type]=current

Next, let's lump the rarity categories. Instead of the granular "singleton, rare, ultrarare..." we want the much more general RARE and COMMON, which each lump multiple categories. 

In [16]:
for cell_type in cell_types:
    current=data[cell_type]
    current["rarity_bin"]=data[cell_type].apply(GC.lump_rarity_categories,axis=1)
    data[cell_type]=current

End redundant cells

In [167]:
OR_figures=[]


#cell-type subset
for cell_type in GC.cell_types:
    ref_bins=data[cell_type]['ref_bin'].unique()
    skew_bins=data[cell_type]["skew_bin"].unique()
    for genome_region in GC.genome_regions:
        for ref_bin in ref_bins:
            for skew_bin in skew_bins:
                
                

                working=data[cell_type]
                if genome_region != "ALL":
                    working=working[working[genome_region]==True]
                working=working[working['ref_bin']==ref_bin]

                if working["count"].sum() < 100000:
                    continue
                
                ret={}
                ret["cell_type"]=cell_type
                ret["genome_region"]=genome_region
                ret["ref_bin"]=ref_bin
                ret["skew_bin"]=skew_bin
                
                
                or_dict=ORC.compute_OR(df=working,
                               a="rarity_bin",
                               a_val="RARE",
                               b="skew_bin",
                               b_val=skew_bin)
                ret=dict(ret, **or_dict)

                OR_figures.append(ret)

/home/mcn26/.conda/envs/mcn_vareff_2/lib/python3.10/site-packages/scipy/stats/_odds_ratio.py:156: RuntimeWarning: divide by zero encountered in divide
  se = np.sqrt((1/table).sum())
/home/mcn26/.conda/envs/mcn_vareff_2/lib/python3.10/site-packages/scipy/stats/_odds_ratio.py:168: RuntimeWarning: invalid value encountered in scalar subtract
  loglow = log_or - z*se
/home/mcn26/.conda/envs/mcn_vareff_2/lib/python3.10/site-packages/scipy/stats/_odds_ratio.py:155: RuntimeWarning: divide by zero encountered in log
  log_or = np.log(oddsratio)
/home/mcn26/.conda/envs/mcn_vareff_2/lib/python3.10/site-packages/scipy/stats/_odds_ratio.py:169: RuntimeWarning: invalid value encountered in scalar add
  loghigh = log_or + z*se


In [168]:
OR_figures=pd.DataFrame(OR_figures).dropna()

In [193]:
for cell_type in GC.cell_types:
    
    
    for genome_region in GC.genome_regions:
        
        if genome_region=="ALL":
            continue
        working=OR_figures
        working = working[working['ref_bin'].str.startswith(cell_type)]
        
        if genome_region != "ALL":
            working=working[working["genome_region"]==genome_region]
        
        
        working['error_length_lower'] = working['OR'] - working['ci_lower']
        working['error_length_upper'] = working['ci_upper'] - working['OR']
        
        #SORT
        
        for row in working.iterrows():
            pass

